# Emotions classification

## Downloads & imports

In [ ]:
!pip install aniemore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179306 sha256=54692feffa3fa54cf4316f1bc14b109f03f3b3f15755b67573126815c6dc23c3
  Stored in directory: /root/.cache/pip/wheels/ed/9c/6d/9df9e633b11625901f05fbf6b7bb89aa365705be1f1aa65c7d
Successfully built openai-whisper
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers

In [ ]:
import torch
import aniemore
import os
import numpy as np
import warnings
import scipy.io.wavfile as wavf
from tqdm import tqdm
from aniemore.recognizers.voice import VoiceRecognizer
from aniemore.models import HuggingFaceModel
from zipfile import ZipFile as zlib
warnings.filterwarnings('ignore')

## RESD

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Aniemore/resd")

Generating train split:   0%|          | 0/1116 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/280 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'path', 'emotion', 'speech'],
        num_rows: 1116
    })
    test: Dataset({
        features: ['name', 'path', 'emotion', 'speech'],
        num_rows: 280
    })
})

In [ ]:
dataset["train"][0]

{'name': '32_happiness_enthusiasm_h_120',
 'path': 'happiness_enthusiasm_32/32_happiness_enthusiasm_h_120.wav',
 'emotion': 'happiness',
 'speech': {'path': '32_happiness_enthusiasm_h_120.wav',
  'array': array([-0.00018311, -0.00061035, -0.00076294, ...,  0.00085449,
          0.00048828,  0.00030518]),
  'sampling_rate': 16000}}

In [ ]:
answers = dataset["train"][:]["emotion"]
answers_test = dataset["test"][:]["emotion"]

In [ ]:
os.mkdir("resd")

In [ ]:
for i, arr in enumerate(dataset["train"]):
    out_f = "resd/" + str(i) + ".wav"
    wavf.write(out_f, 16000, arr["speech"]["array"])

In [ ]:
os.mkdir("resd_test")

In [ ]:
for i, arr in enumerate(dataset["test"]):
    out_f = "resd_test/" + str(i) + ".wav"
    wavf.write(out_f, 16000, arr["speech"]["array"])

## Models

In [ ]:
[e.value for e in HuggingFaceModel.Voice]

[Model(model_cls=<class 'transformers.models.wav2vec2.modeling_wav2vec2.Wav2Vec2ForSequenceClassification'>, model_url='aniemore/wav2vec2-emotion-russian-resd'),
 Model(model_cls=<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, model_url='aniemore/wav2vec2-xlsr-53-russian-emotion-recognition'),
 Model(model_cls=<class 'transformers.models.wavlm.modeling_wavlm.WavLMForSequenceClassification'>, model_url='aniemore/wavlm-emotion-russian-resd'),
 Model(model_cls=<class 'transformers.models.hubert.modeling_hubert.HubertForSequenceClassification'>, model_url='aniemore/hubert-emotion-russian-resd'),
 Model(model_cls=<class 'transformers.models.unispeech_sat.modeling_unispeech_sat.UniSpeechSatForSequenceClassification'>, model_url='aniemore/unispeech-sat-emotion-russian-resd')]

In [ ]:
def get_acc(model):
    acc = 0
    size = len(answers)
    for i in tqdm(range(len(answers))):
        if answers[i] == "fear": # extra
            size -= 1
            continue
        name = "resd/" + str(i) + ".wav"
        res = model.recognize(name, return_single_label=True)
        acc += res == answers[i]
    return acc, size

### WavLM

In [ ]:
wavlm = VoiceRecognizer(model=HuggingFaceModel.Voice.WavLM)

config.json:   0%|          | 0.00/2.91k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [ ]:
acc_wavlm, size = get_acc(wavlm)

100%|██████████| 1116/1116 [3:07:38<00:00, 10.09s/it]


In [ ]:
print(size)
acc_wavlm / size

938


0.5479744136460555

### Hubert

In [ ]:
hubert = VoiceRecognizer(model=HuggingFaceModel.Voice.Hubert)

In [ ]:
acc_hubert, size = get_acc(hubert)

100%|████████████████████████████████████████████████████████████████████████████| 1116/1116 [1:29:40<00:00,  4.82s/it]


In [ ]:
print(size)
acc_hubert / size

938


0.5597014925373134

### Wav2Vec2

In [ ]:
wav2vec2 = VoiceRecognizer(model=HuggingFaceModel.Voice.Wav2Vec2)

In [ ]:
acc_wav2vec2, size = get_acc(wav2vec2)

100%|████████████████████████████████████████████████████████████████████████████| 1116/1116 [1:35:15<00:00,  5.12s/it]


In [ ]:
print(size)
acc_wav2vec2 / size

938


0.5660980810234542

### UniSpeech

In [ ]:
uni_speech = VoiceRecognizer(model=HuggingFaceModel.Voice.UniSpeech)

In [ ]:
acc_uni_speech, size = get_acc(uni_speech)

100%|████████████████████████████████████████████████████████████████████████████| 1116/1116 [1:36:35<00:00,  5.19s/it]


In [ ]:
print(size)
acc_uni_speech / size

938


0.5543710021321961

## RAVDESS (English)

Радость - 03 - happiness

Грусть - 04 - sadness

Злость - 05 - anger

Удивление - 08

Нейтрально - 01 (02 - спокойствие) - neutral

Отвращение - 07 - disgust

In [ ]:
with zlib("ravdess.zip", 'r') as z:
    z.extractall()
    print("Done")

Done


In [ ]:
model = HuggingFaceModel.Voice.WavLM
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
vr = VoiceRecognizer(model=model)

C:\Users\irish\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\irish\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at aniemore/wavlm-emotion-russian-resd were not used when initializing WavLMForSequenceClassification: ['wavlm.e

Some weights of the model checkpoint at aniemore/wavlm-emotion-russian-resd were not used when initializing WavLMForSequenceClassification: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForSequenceClassification were not initialized from the model checkpoint at aniemore/wavlm-emotion-russian-resd and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wavlm.encoder.pos_conv_embed.conv.param

In [ ]:
res = vr.recognize('ravdess/03-01-03-02-01-01-01.wav')

C:\Users\irish\anaconda3\Lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [ ]:
res

{'anger': 0.041953977197408676,
 'disgust': 0.00017660057346802205,
 'enthusiasm': 0.000249645730946213,
 'fear': 0.0046153864823281765,
 'happiness': 0.9528968334197998,
 'neutral': 6.682628009002656e-05,
 'sadness': 4.0788618207443506e-05}

In [ ]:
ravdess_emotions = {
    1: "neutral",
    3: "happiness",
    4: "sadness",
    5: "anger",
    7: "disgust",
    8: "surprise"
}

In [ ]:
res = {
    "anger": 0.0,
    "disgust": 0.0,
    "enthusiasm": 0.0,
    "fear": 0.0,
    "happiness": 0.0,
    "neutral": 0.0,
    "sadness": 0.0
}

In [ ]:
use = {"01", "03", "04", "05", "07"}

In [ ]:
len(os.listdir('ravdess'))

1440

In [ ]:
cnt = 0
acc = 0
for file in os.listdir('ravdess'):
    if file.split('-')[2] in use:
        cnt += 1
        #print(file)
        #print(vr.recognize('ravdess/' + file))
        #print(vr.recognize('ravdess/' + file, return_single_label=True) == ravdess_emotions[int(file.split('-')[2])])
        acc += vr.recognize('ravdess/' + file, return_single_label=True) == ravdess_emotions[int(file.split('-')[2])]
        if cnt % 100 == 0:
            print(cnt)

100
200
300
400
500
600
700
800


In [ ]:
print(acc)
print(acc / cnt)

263
0.30439814814814814
